<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2016</h1>
<h1 style="font-size: 250%;">Problem Set #1 </h1>
<h1>Issued Wednesday, 2/1/2017; Due Monday, 2/13/2017</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [8]:
# Replace the following string values with the requested information
class Student:
    first = "Zhilan"
    last = "Zhou"
    onyen = "zzl"
    pid = "720432337"

---
**Problem #1:** Given a 1.1 million base-pair genome, what do you expect is the length of the shortest *missing* *k*-mer in the genome (i.e. it does not appear)? What are the assumptions of your answer, if any? How many times would you expect to see a given 9-base subsequence appear in the same genome, assuming all bases are independent and equally likely?

---
**Problem #2:** Find the shortest missing *k*-mer or *k*-mers on chromosome 1 of [Vibrio cholerae](http://csbio.unc.edu/mcmillan/Comp555S17/data/VibrioCholerae.fa). If there are more than one, report all of them.

---
**Problem #3:** Modify the <code>kmerPositions()</code> function given in Lecture 2 on slide 20 to allow for errors. Your new function:

     kmersWithErrors(k, sequence, errorCount)

should return a Python dictionary whose keys are kmers that appear in either the sequence or its reverse complement, and whose values are a list of positions of a kmer that differs in no more than ***errorCount*** bases from the key or its reverse complement. Your list should include only the lexigraphically first of each kmer/reverse complement pair. Enter the code for your function in the space provided below.

You can test your code on the following sequence:
<pre>
test = "atcaatgatcaacgtaagcttctaagcatgatcaaggtgctcacacagtttatccacaac"+\
       "ctgagtggatgacatcaagataggtcgttgtatctccttcctctcgtactctcatgacca"+\
       "cggaaagatgatcaagagaggatgatttcttggccatatcgcaatgaatacttgtgactt"+\
       "gtgcttccaattgacatcttcagcgccatattgcgctggccaaggtgacggagcgggatt"+\
       "acgaaagcatgatcatggctgttgttctgtttatcttgttttgactgagacttgttagga"+\
       "tagacggtttttcatcactgactagccaaagccttactctgcctgacatcgaccgtaaat"+\
       "tgataatgaatttacatgcttccgcgacgatttacctcttgatcatcgatccgattgaag"+\
       "atcttcaattgttaattctcttgcctcgactcatagccatgatgagctcttgatcatgtt"+\
       "tccttaaccctctattttttacggaagaatgatcaagctgctgctcttgatcatcgtttc"
</pre>
But a different sequence will be used for grading.

In [ ]:
def kmersWithErrors(k, sequence, errorCount):
    kmerPosition = {}
    for i in xrange(0,len(sequence)-k+1):
        kmer = sequence[i:i+k]
        kmerPosition[kmer] = kmerPosition.get(kmer,[])+[i]
    # combine kmers and their reverse complements
    
    srev = ''.join([{'A':'T','C':'G','G':'C','T':'A'}[base] for base in reversed(sequence)])
    for i in xrange(0, len(srev)-k+1):
        kmer = srev[i:i+k]
        kmerPosition[kmer] = kmerPosition.get(kmer,[])+[-k-i]
    
    kmerList = kmerPosition.keys()
    N = len(kmerList)
    kmersWithErrors = {kmer: pos for kmer, pos in kmerPosition.iteritems()}
    for i in xrange(N-1):
        for j in xrange(i+1, N):
            hammingDist = sum([1 if kmerList[i][w] != kmerList[j][w] else 0 for w in xrange(k)])
            if (hammingDist <= errorCount):
                kmersWithErrors[kmerList[i]] += kmerPosition[kmerList[j]]
                kmersWithErrors[kmerList[j]] += kmerPosition[kmerList[i]]
    pairPosition = {}
    for element in kmersWithErrors.iterkeys():
        revElement = ''.join([{'A':'T','C':'G','G':'C','T':'A'}[base] for base in reversed(element)])
        if ((element not in pairPosition) and (revElement not in pairPosition)):
            if (element < revElement):
                if (revElement in kmersWithErrors):
                    pairPosition[element] = kmersWithErrors[element] + kmersWithErrors[revElement]
                else:
                    pairPosition[element] = kmersWithErrors[element]
            elif (element == revElement):
                pairPosition[kmer] = kmersWithErrors[element]
            else:
                if revElement not in kmerPosition:
                    pairPosition[revElement] = kmersWithErrors[element]
    return pairPosition

**Problem #4:** A restriction enzyme is a protein produced by bacteria that cleaves DNA at specific subsequence patterns called *recognition sites*. Bacteria use restriction enzymes to defend against bacterial viruses called bacteriophages. The recognition site of a restriction enzyme is commonly a *reverse palindrome*, meaning the base sequence reads the same backwards and forwards, with a length of 4, 6, or 8 bases. The intervals between recognition sites are called *digestion fragments*.

How many possible reverse palindromic recognition sites of length 8 exist? What fraction of 8-mers are reverse palindromic? Approximately how frequently, in terms of base-pairs, would you expect a given 8-base recogonition site might appear in a genome? How frequently does the recognition site GAATTC appear in chromosome 1 of [Vibrio cholerae](http://csbio.unc.edu/mcmillan/Comp555S17/data/VibrioCholerae.fa)? How many digestion fragments result?

**Problem #5: Programming Problem**

Write a branch-and-bound version of the *MedianStringMotifSearch()* algorithm shown below. You should base your alogrithm on the observation that, if the best alignment of a *prefix* of a candidate motif has a greater Hamming distance than the minimal Hamming distance of the best motif found so far, it could not possibly lead to a better solution.

<pre>
import itertools

def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in xrange(len(seq)-k+1):
            HammingDist = sum([1 for i in xrange(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def MedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('acgt', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer
</pre>

Apply your algoritm to the set of sequences used in lecture:

<pre>
seqApprox = [
    'tagtggtcttttgagtgtagatctgaagggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat',
    'cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagttggatccgaaactggagtttaatcggagtcctt',
    'gttacttgtgagcctggttagacccgaaatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt',
    'aacatcaggctttgattaaacaatttaagcacgtaaatccgaattgacctgatgacaatacggaacatgccggctccggg',
    'accaccggataggctgcttattaggtccaaaaggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac',
    'tagattcgaatcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc',
    'gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtgcagatccgaacgtctctggaggggtcgtgcgcta',
    'atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtgtagatccgta',
    'ttcttacacccttctttagatccaaacctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac',
    'ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacaggtcgatccgaaattcg']
</pre>

Use the %time *magic* function to compare the performance of your method to the one given.

In [2]:
import itertools
global cDNA, kmerList, bestScoreAll, bestScore, bestAlignmentAll
    
def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in xrange(len(seq)-k+1):
            HammingDist = sum([1 for i in xrange(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def generateKmers(k, prefix=''):
    global kmerList, bestScore, bestScoreAll, bestAlignmentAll
    if (prefix == ''):
        kmerList = []
    if (len(prefix) == k):
        currentAli, currentScore = ScanAndScoreMotif(cDNA, prefix)
        if (currentScore <= bestScoreAll):
            bestScoreAll = currentScore
            bestAlignmentAll = currentAli
            kmerList.append(prefix)
        bestScore = k*len(cDNA)
    else:
        currentAli, currentScore = ScanAndScoreMotif(cDNA, prefix)
        currentScore += (k-len(prefix))*len(cDNA)
        if (currentScore <= bestScore):
            bestScore = currentScore
            for base in "acgt":
                generateKmers(k, prefix+base)
                
def BranchAndBoundMedianStringMotifSearch(DNA,k):
    global cDNA, bestScore, bestScoreAll, kmerList
    cDNA = DNA
    bestScore = k*len(cDNA)
    bestScoreAll = k*len(cDNA)
    generateKmers(k)
    minHammingDist = bestScoreAll
    for pattern in kmerList:
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        
        if (dist == minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            bestKmer = motif
    return bestAlignmentAll, bestScoreAll, bestKmer

%time BranchAndBoundMedianStringMotifSearch(seqApprox,10)

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set